In [1]:
!pip install -q pyomo
!pip install -q folium
!apt install -y -q coinor-cbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.4 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 21 not upgraded.
Need to get 2,875 kB of archives.
After this operation, 8,782 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-1 [480 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/univer

In [2]:
import numpy as np
import pandas as pd
import folium
from pyomo.environ import (ConcreteModel, 
                           Var, 
                           ConstraintList, 
                           Objective, 
                           Binary, 
                           PositiveIntegers,
                           SolverFactory)

# initialize solver
solver = SolverFactory("cbc", executable='/usr/bin/cbc')

In [3]:
cities_data = pd.read_csv("/content/cities_data.csv", index_col=0)
cities_data.head()

,id,name,lat,lon,location
0,1997159,Sangli,17.172693,74.586765,"Sangli, Maharashtra, India"
1,1986143,Satara,17.636129,74.298278,"Satara, Maharashtra, India"
2,593878343,Kolhapur,16.702841,74.240533,"Kolhapur, Maharashtra, 416003, India"
3,10351626,Pune,18.521428,73.854454,"Pune City, Maharashtra, India"
4,1991136,Akola,20.761812,77.192116,"Akola, Maharashtra, India"


In [4]:
cities_distances = pd.read_csv("cities_distances.csv", index_col=0)
cities_distances.head()

,Sangli,Satara,Kolhapur,Pune,Akola,Amravati,Yavatmal,Aurangabad,Jalna,Osmanabad,...,Nagpur,Wardha,Nandurbar,Nashik,Ahmednagar,Palghar,Raigad,Ratnagiri,Sindhudurg,Mumbai
Sangli,0.000000,59.805481,63.780870,168.446392,482.982288,545.708113,509.790596,310.331413,333.276599,196.064251,...,645.396956,585.311760,481.750322,325.885130,222.644290,334.756249,211.810446,120.346004,152.004055,277.790454
Satara,59.805481,0.000000,103.729577,108.808537,460.623738,524.360683,499.516901,271.628983,302.370351,201.036350,...,634.432954,573.995387,431.052917,268.866460,179.339490,275.031509,154.954591,97.298340,180.047509,219.027934
Kolhapur,63.780870,103.729577,0.000000,205.862033,546.751260,609.488516,572.550017,370.721411,395.976470,256.871799,...,708.124794,648.162649,534.782446,370.118565,280.604954,362.917287,230.291053,105.118957,88.766991,300.274658
Pune,168.446392,108.808537,205.862033,0.000000,428.254490,491.680361,488.838685,216.389221,262.034167,241.542260,...,618.561905,558.983988,339.699218,165.433112,127.153953,168.246467,75.439398,143.711054,265.560940,119.729042
Akola,482.982288,460.623738,546.751260,428.254490,0.000000,63.981328,107.577097,216.344302,166.254232,308.748319,...,200.494766,147.564023,286.803537,363.457867,301.193676,470.095645,492.774914,549.942917,634.150633,487.180246


In [5]:
def get_map(cities_data, cities_distances, polylines=None, distances=None):
    cities_coords = cities_data.iloc[:, 2:4].to_numpy()
    cities_names = cities_data.iloc[:, 1].to_numpy()
    cities_dists = cities_distances.to_numpy()

    min_pt = (cities_data[['lat', 'lon']].min().values - 2).tolist()
    max_pt = (cities_data[['lat', 'lon']].max().values + 2).tolist()
    
    mymap = folium.Map(location=[28.5011226, 77.4099794])
    for name, pt in zip(cities_names, cities_coords):
        folium.Marker(list(pt), popup = name).add_to(mymap)
    mymap.fit_bounds([min_pt, max_pt])
    if polylines is not None:
        for i in range(len(polylines)-1):
            line = tuple([polylines[i], polylines[i+1]])
            if distances is not None:
                folium.PolyLine(line, tooltip=distances[i], popup=distances[i], weight=6, color="#111").add_to(mymap)
            else:
                folium.PolyLine(line, weight=1, color="#111").add_to(mymap)
    return mymap

In [6]:
mymap = get_map(cities_data, cities_distances)
mymap

In [7]:
cities_coords = cities_data.iloc[:, 2:4].to_numpy()
cities_names = cities_data.iloc[:, 1].to_numpy()
distance_matrix = cities_distances.to_numpy()
print("shape of distance matrix: ", distance_matrix.shape)

shape of distance matrix:  (25, 25)


In [8]:
def TSPModel(distance_matrix):
    n = len(distance_matrix)
    model = ConcreteModel(name="TSPSolver")

    model.x = Var(range(n), range(n), initialize=0.0, domain=Binary)
    model.u = Var(range(n), domain=PositiveIntegers)

    model.objective = Objective(expr=sum(distance_matrix[i, j]*model.x[i, j] for i in range(n) for j in range(n)))

    model.constraints = ConstraintList()
    for i in range(n):
        model.constraints.add(sum(model.x[i, k] for k in range(n) if i != k) == 1)

    for j in range(n):
        model.constraints.add(sum(model.x[k, j] for k in range(n) if j != k) == 1)

    model.constraints.add(model.u[0] == 1)
    for i in range(1, n):
        model.u[i].setlb(2)
        model.u[i].setub(n)

    for i in range(n):
        for j in range(n):
            if i != 0 and j != 0:
                model.constraints.add(model.u[i] - model.u[j] + 1 <= (n-1)*(1-model.x[i, j]))
    return model

In [9]:
model = TSPModel(distance_matrix)
model.pprint()

5 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  627 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 

In [10]:
%%time
result = solver.solve(model)
print("Solver Status: ", result.solver.status)
print("Solver Termination Condition: ", result.solver.termination_condition)

Solver Status:  ok
Solver Termination Condition:  optimal
CPU times: user 571 ms, sys: 81.7 ms, total: 652 ms
Wall time: 48.8 s


In [11]:
print("optimal objective value: ", model.objective())

optimal objective value:  2776.650165817489


In [12]:
optimal_path_idx = [int(model.u[i].value) for i in range(len(distance_matrix))]
optimal_path = [cities_names[i-1] for i in optimal_path_idx]
for city in optimal_path:
    print(city, " -> ", end="")
print(cities_names[0])

Sangli  -> Satara  -> Mumbai  -> Kolhapur  -> Gondia  -> Chandrapur  -> Jalna  -> Wardha  -> Nagpur  -> Akola  -> Yavatmal  -> Amravati  -> Aurangabad  -> Osmanabad  -> Nanded  -> Latur  -> Hingoli  -> Nandurbar  -> Nashik  -> Pune  -> Ahmednagar  -> Raigad  -> Ratnagiri  -> Sindhudurg  -> Palghar  -> Sangli


In [13]:
polylines = [cities_coords.tolist()[i-1] for i in optimal_path_idx] + [cities_coords.tolist()[optimal_path_idx[0]-1]]

In [14]:
distances = []
for i in range(len(optimal_path_idx)-1):
    distances.append(f"{distance_matrix[optimal_path_idx[i]-1, optimal_path_idx[i+1]-1] :.2f} K.M.")
distances.append(f"{distance_matrix[optimal_path_idx[-1]-1, optimal_path_idx[0]-1] :.2f} K.M.")

In [15]:
mymap = get_map(cities_data, cities_distances, polylines, distances)
mymap